In [3]:
#Converting TeleQNA.txt to dictionary format
import ast
import json
with open("TeleQnA.txt", "r") as f:
    content = f.read()

nested_dict = ast.literal_eval(content)


result_dict = {}
for item in nested_dict:
    question_num = item
    question = nested_dict.get(item).get("question")
    answer = nested_dict.get(item).get("answer")[10:]
    print(answer)
    explanation = nested_dict.get(item).get("explanation")
    result_dict[question_num] = {
        "question": question,
        "answer": answer,
        "explanation": explanation
    }

with open("TeleQnA.json", "w") as f:
    json.dump(result_dict, f,indent=4)


To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints
SCMA
2
All devices
It uses information stored in the UE when it was detached.
MBR point and MSR point
2-5 km
Network Function Virtualization (NFV)
Maximizing system throughput
Virtual Private Network
Independence and uniformity
Anytime
DNAI(s) or the updated DNAI information
H(X) = E(log(1/p(X)))
By the OCF
To collimate waves from a feed into a pencil beam
Low latency
Total Radiated Power
Waves where both the electric field and the magnetic field are perpendicular to the direction of propagation
To transmit HARQ feedback for unicast and groupcast
The number of chips used to transmit a single bit
To minimize buffering latency at the receivers
Downlink Reference Signal power.
WLAN Offloading System
DR8/10 and DR9/11
Ubiquitous connectivity to diversified user and device types
8
Unsupervised pre-trained networks, convolutional neural networks, and recursive neural networks
ener

## Step 0
### import

In [4]:
from datasets import load_dataset, DatasetDict, Dataset



## Step 1
### Import TeleQNA into a list

#### Example 1:
##### [{'question 1': 'What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]',
##### 'answer':'To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints'},
##### 'explaination': The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints. ']

In [1]:
import json

with open("TeleQnA.json", "r") as f:
    data_dict = json.load(f)

data_list = []
for item in data_dict:
    question = data_dict[item]["question"]
    answer = data_dict[item]["answer"]
    explanation = data_dict[item]["explanation"]
    data_list.append({
        "input": question,
        "output": "The correct answer is : "+ answer + ". Because: " +  explanation,
    })


## Step 2:
### Load Model & Tokenizer

In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer


model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

d:\lc\venv\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 3:
### Tokenize Dataset

In [5]:
def tokenize(examples):
    input_text = examples["input"]
    target_text = examples["output"]
    inputs = tokenizer(input_text, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(target_text, max_length=512, truncation=True, padding="max_length")
    inputs['labels'] = labels["input_ids"]
    return inputs


dataset = Dataset.from_list(data_list)
dataset = dataset.map(tokenize)



Map: 100%|██████████| 10000/10000 [00:07<00:00, 1350.13 examples/s]


## Step 3:
### Tokenize Dataset

In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=1,
    logging_steps=100,
    save_strategy="no",
    evaluation_strategy="no",
    fp16=False,
    bf16=False,
    gradient_checkpointing=False,
    disable_tqdm=False,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model("my_model_output")
tokenizer.save_pretrained("my_model_output")


d:\lc\venv\llm\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\lc928\AppData\Local\Temp\ipykernel_864\2823558702.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/2500 [00:00<?, ?it/s]d:\lc\venv\llm\lib\site-packages\transformers\models\bart\modeling_bart.py:497: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  4%|▍         | 100/2500 [00:40<15:18,  2.61it/s]

{'loss': 8.0502, 'grad_norm': 195.56190490722656, 'learning_rate': 9.600000000000001e-06, 'epoch': 0.04}


  8%|▊         | 200/2500 [01:20<15:44,  2.44it/s]

{'loss': 2.9063, 'grad_norm': 258.0140686035156, 'learning_rate': 9.200000000000002e-06, 'epoch': 0.08}


 12%|█▏        | 300/2500 [02:01<14:59,  2.45it/s]

...

 96%|█████████▌| 2400/2500 [14:00<00:32,  3.10it/s]

{'loss': 0.1717, 'grad_norm': 2.090050220489502, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.96}


100%|██████████| 2500/2500 [14:35<00:00,  2.86it/s]
d:\lc\venv\llm\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 0.1858, 'grad_norm': 2.863924026489258, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 875.0576, 'train_samples_per_second': 11.428, 'train_steps_per_second': 2.857, 'train_loss': 0.6489118194580078, 'epoch': 1.0}


('my_model_output\\tokenizer_config.json',
 'my_model_output\\special_tokens_map.json',
 'my_model_output\\vocab.json',
 'my_model_output\\merges.txt',
 'my_model_output\\added_tokens.json')

In [11]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("my_model_output")
tokenizer = BartTokenizer.from_pretrained("my_model_output")

prompt = "What are the extreme points of the family of Regenerating Codes?"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)



The correct answer is : All of the above. Because: The extreme points of the family of Regenerating Codes are the number of cells in a cell, the number in the cell, and the number at the end of the cell.
